In [1]:
# 看資料

import json
with open("/user_data/preprocess/wikidata/data/AA/wiki_01","r",encoding="UTF-8") as f:
    data = f.readlines()
    c = 0
    for line in data:
        article = json.loads(line)
        length = len(article['text'])
        if length > 256: 
            print(c, "---------------------------------")
            print(article)
            print(f'length of text: {length}')
            c+=1
            
        else: continue
        
        if c == 1:
            break
        

0 ---------------------------------
{'id': '118', 'revid': '2146037', 'url': 'https://zh.wikipedia.org/wiki?curid=118', 'title': '中华人民共和国', 'text': "中华人民共和国，简称中国，是一个位於东亚的社会主义国家，成立于1949年10月1日，首都为北京市。中华人民共和国领土陸地面積約960萬平方公里，是世界上陸地面積第三大的國家，一共划分為23個省份、5個自治區、4個直轄市和2個特別行政區。中华人民共和国地势西高东低而呈現三级阶梯分布，大部分地区属于溫帶、副熱帶季风气候，地理景致與氣候型態丰富多樣，有冰川、丹霞、黃土、沙漠、喀斯特等多种地貌，中华人民共和国北方分布有乾草原和荒漠，南方有热带雨林，西部至西南部則有天山山脈、帕米尔高原、青藏高原、喀喇崑崙山脈和喜馬拉雅山脈，东临太平洋。领海由渤海（内海）以及黄海、东海、南海三大边海组成，水域面积约470万平方千米，分布有大小岛屿7,600个。中华人民共和国主張领土東至黑龙江省抚远市的黑瞎子岛中部，西至新疆境内的帕米尔高原，北至黑龙江省大兴安岭地区的漠河县，南至南海曾母暗沙。\n中华人民共和国是目前世界上人口最多的国家，2020年約有14.2億人，同時也是一个多民族国家，官方承認的民族共有56個，其中汉族佔91.51%总人口。现代标准汉语普通话和规范汉字为国家通用语言和文字，在民族自治地方其他民族可以使用民族語言和文字。自1986年起实行九年义务教育制度，就读公立学校的学生由政府提供其学费。2020年人口普查显示，全国人口文盲率为2.67%。接受高等教育人口达2.4亿。\n中华人民共和国目前为世界第二大经济体，2021年國內生產總值（）总量为17.7万亿美元，按照國際匯率计算位居世界第二，仅次于美国，按購買力平價则位列世界第一。1978年改革開放後，中华人民共和国成为經濟成長最快的主要經濟體之一，GDP实际增长率在经济总量前20名的国家中常年位居第一。当前，中華人民共和國对外贸易额世界第一，是世界上最大的商品出口國及第二大的進口國。依据國際匯率計算，中国大陆人均GDP从1990年的347美元增长至2021年近12,600美元（IMF数据），增长近35倍，东部地区部分城市人均GDP已超2

In [17]:
# remove duplicate item

def remove_duplicates(lst: list) -> list:
    seen = set()
    result = []
    for item in lst:
        if item not in seen:
            result.append(item)
            seen.add(item)
    return result

In [18]:
# split text into some chunks

def split_list(text: str, chunk_size: int) -> list:
    result = []

    for i in range(0, len(text), chunk_size//2):
        chunk = text[i:i + chunk_size]
        # print(f'i = {i} which is {text[i]}')
        
        if len(chunk) < chunk_size: # complete the end string
            chunk = text[-chunk_size:]    
           
        result.append(chunk)
    
    return remove_duplicates(result)

In [4]:
# split_list功能測試

a = 'abcdefghijklmnopqrstuvwxyz'
print(split_list(a, 6))


['abcdef', 'defghi', 'ghijkl', 'jklmno', 'mnopqr', 'pqrstu', 'stuvwx', 'uvwxyz']


In [19]:
# extract data from give data path

import json
import os
from tqdm.auto import tqdm
from opencc import OpenCC

converter = OpenCC('s2t')

def extract_data(file_path: str, slice: int):
    preprocess_file = []
    with open(file_path, "r",encoding="UTF-8") as f:
        data = f.readlines()
        
        for line in data:
            article = json.loads(line)
            text = article['text']
            text_list = []
            if len(text) > 128:
                text_list = split_list(text, slice)
                text_list = [converter.convert(text) for text in text_list]
                article['title'] = converter.convert(article['title'])
                
                # print(article)
                # print(text_list)
                # print(f"length of text {int(article['id'])}: {len(text_list[-1])}")
                # print()
                
                for i in range(len(text_list)):
                    
                    # 建反向索引必須包含id和contents!!!!!
                    preprocess_file.append({
                        'id': article['id'],
                        'inner_id': int(i),
                        'contents': text_list[i]
                    })
                    
                    
    return preprocess_file

In [21]:
# process all files and generate json files

def process_files_in_directory(file_path: str, slice: int):
    # file_path = '/user_data/preprocess/wikidata/data'
    output_path = '/user_data/preprocess/wikijson'
    output_path = output_path + f'_{slice}'
    
    dataset = []
    
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    
    with tqdm(total=len(os.listdir(file_path))) as pbar:
        for root, dirs, files in os.walk(file_path, topdown=False):
            for file_name in files:
                file_path = os.path.join(root, file_name)
                dataset.extend(extract_data(file_path=file_path, slice=slice))

                pbar.update(1) # update progress bar
            
    with open(output_path+f'/wiki_process_{slice}.json', 'w', encoding='UTF-8') as output:
        json.dump(dataset, output, indent=4, ensure_ascii=False)

In [22]:
if __name__ == '__main__':
    file_path = '/user_data/preprocess/wikidata'
    
    for slice in [256, 512, 1024]:
        print('-'*10 + f'Processing Slice {slice}' + '-'*10)
        process_files_in_directory(file_path=file_path, slice=slice)

----------Processing Slice 256----------


3548it [46:03,  1.28it/s]                    


----------Processing Slice 512----------


3548it [41:51,  1.41it/s]                    


----------Processing Slice 1024----------


3548it [38:03,  1.55it/s]                    


In [ ]:
!sudo apt-get update

In [ ]:
!sudo apt-get install openjdk-11-jdk

In [36]:
! python -m pyserini.index.lucene \
  --collection JsonCollection \
  --input /user_data/preprocess/wiki_process/wikijson_1024 \
  --language zh \
  --index /user_data/preprocess/wiki_index/wiki_index_1024 \
  --generator DefaultLuceneDocumentGenerator \
  --threads 1 \
  --storePositions --storeDocvectors --storeRaw

2023-11-29 14:14:00,217 INFO  [main] index.IndexCollection (IndexCollection.java:380) - Setting log level to INFO
2023-11-29 14:14:00,218 INFO  [main] index.IndexCollection (IndexCollection.java:383) - Starting indexer...
2023-11-29 14:14:00,218 INFO  [main] index.IndexCollection (IndexCollection.java:384) - ============ Loading Parameters ============
2023-11-29 14:14:00,218 INFO  [main] index.IndexCollection (IndexCollection.java:385) - DocumentCollection path: /user_data/preprocess/wiki_process/wikijson_1024
2023-11-29 14:14:00,219 INFO  [main] index.IndexCollection (IndexCollection.java:386) - CollectionClass: JsonCollection
2023-11-29 14:14:00,219 INFO  [main] index.IndexCollection (IndexCollection.java:387) - Generator: DefaultLuceneDocumentGenerator
2023-11-29 14:14:00,219 INFO  [main] index.IndexCollection (IndexCollection.java:388) - Threads: 1
2023-11-29 14:14:00,219 INFO  [main] index.IndexCollection (IndexCollection.java:389) - Language: zh
2023-11-29 14:14:00,219 INFO  [ma

In [40]:
from pyserini.search.lucene import LuceneSearcher
import time
searcher = LuceneSearcher('/user_data/preprocess/wiki_index/wiki_index_1024')

start = time.time()
hits = searcher.search('數論',  k=100000)
end = time.time()

# for i in range(len(hits)):
#     print(f'{i+1:2} {hits[i].docid:4} {hits[i].score:.5f}')

print("執行時間：%f 秒" % (end - start))

執行時間：0.056029 秒


In [54]:
from pyserini.search.lucene import LuceneSearcher
searcher = LuceneSearcher('/user_data/preprocess/wiki_index/wiki_index_1024')
searcher.set_language('zh')
hits = searcher.search('習近平生日什麼時候',k=10)

import json
top_article_raw = json.loads(hits[0].raw)
print(top_article_raw)
top_article = top_article_raw['contents']
# print(top_article)


# print(top_article.find('習近平'))

{'id': '3167269', 'inner_id': 1, 'contents': '1983年3月，加入中國共產黨。\n1986年7月從北京師範大學畢業後，到中共中央辦公廳辦公室工作，任主任科員，副處長。\n1989年9月起，調入新組建的中央政策研究室工作，歷任處長，黨建組副組長（副局級）、組長（正局級）。\n2000年6月，任中共中央政策研究室副主任，中央黨建工作領導小組成員兼祕書組組長。 2005年，兼任中央保持共產黨員先進性教育活動領導小組成員。2008年，兼任中央深入學習實踐科學發展觀活動試點工作領導小組成員。\n2009年4月，明確爲中央政策研究室常務副主任（正部長級）。任中央政策研究室副主任、常務副主任期間，何毅亭多次陪同胡錦濤、習近平外出調研。\n2013年9月，任中共中央黨校常務副校長。\n2018年3月，任第十三屆全國人民代表大會社會建設委員會主任委員，中共中央黨校（國家行政學院）分管日常工作的副校長（副院長）。\n2020年12月，卸任中央黨校（國家行政學院）常務副校（院）長職務。\n言論.\n2020年6月15日，中共中央總書記習近平生日當天，何毅亭發表專欄文章，稱習近平新時代中國特色社會主義思想是21世紀的馬克思主義，中國的發展超越了國家地理邊界，具有全球歷史意義。文章並將習近平與馬克思相提並論。\n政治評論家陳破空表示，何毅亭是習近平嫡系親信，有人把他稱爲習近平的文膽，除了王滬寧和江金權之外，何毅亭是其中之一。但是何毅亭在二十大前發了一篇文章，就是談戰略問題。在談戰略問題的時候，假裝引用習近平的一些論述，也假裝引用毛澤東的一些說法，什麼全域性性的問題要站得高。但是他在其中卻罕見的反思文革，兩次提到文革不好，文革不對。由習近平的親信說出來，這很罕見，讓人們大喫一驚。因爲習家軍一再放風，說閉關鎖國是好的，能防範敵對勢力。還有計劃經濟有道理，還有階級鬥爭沒有熄滅。還有什麼軍國主義，就是全民要崇軍尚武等等。給人的感覺是文化大革命要定調，肯定在連任之後搞文革。但是沒想到何毅亭的這篇文章表面上來還引了習近平的話，但事實上卻在否定文革。這並不是他一人想這麼做，他也不敢這麼做。一定是習家軍看到頂不住了，或者說習近平在黨內路線鬥爭失敗了。\n科研.\n何毅亭主要關注黨建理論與政府決策研究領域，其主要著作有：《領導者公共形象藝術》、《現代領導哲學思維：